## Scraping Twitch Chat Data

#### Using Twitch Sockets from https://www.learndatasci.com/tutorials/how-stream-text-data-twitch-sockets-python/

In [5]:
import os

server = 'irc.chat.twitch.tv'
port = 6667
nickname = 'gCapstone1'
token = os.environ.get('TWITCH_OAUTH_TOKEN')


#specify the channel and log file name
channel = '#disguisedtoast' #hearthstone
log_file_name = 'disguisedtoast_chat.log'

In [6]:
#To establish a connection to Twitch IRC we'll be using Python's socket library. First we need to instantiate a socket:

import socket

sock = socket.socket()
sock.connect((server, port))

In [7]:
#Once connected, we need to send our token and nickname for authentication, and the channel to connect to over the socket.

# With sockets, we need to send() these parameters as encoded strings:

sock.send(f"PASS {token}\n".encode('utf-8'))
sock.send(f"NICK {nickname}\n".encode('utf-8'))
sock.send(f"JOIN {channel}\n".encode('utf-8'))

21

In [8]:
# Now we have successfully connected and can receive responses from the channel we subscribed to. To get a single response we can call .recv() and then decode the message from bytes:

resp = sock.recv(2048).decode('utf-8')
print(resp)

:tmi.twitch.tv NOTICE * :Improperly formatted auth



In [35]:
# Note: running this the first time will show a welcome message from Twitch. Run it again to show the first message from the channel.

# The 2048 is the buffer size in bytes, or the amount of data to receive. The convention is to use small powers of 2, so 1024, 2048, 4096, etc. Rerunning the above will receive the next message that was pushed to the socket.

resp = sock.recv(2048).decode('utf-8')
print(resp)

:burnout!burnout@burnout.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog



In [36]:
# Writing messages to a file
# Right now, our socket is being inundated with responses from Twitch but we have two problems:

# We need to continuously check for new messages
# We want to log the messages as they come in
# To fix, we'll use a loop to check for new messages while the socket is open and use Python's logging library to log messages to a file.

# First, let's set up a basic logger in Python that will write messages to a file:

import logging

# saves to the specified log file name
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s — %(message)s',
                    datefmt='%Y-%m-%d_%H:%M:%S',
                    handlers=[logging.FileHandler(log_file_name, encoding='utf-8')])

In [37]:
logging.info(resp)

In [ ]:
# Opening chat.log we can see the first message:

# 2018-12-10_11:26:40 — :spappygram!spappygram@spappygram.tmi.twitch.tv PRIVMSG #ninja :Chat, let Ninja play solos if he wants. His friends can get in contact with him.

# So we have the time the message was logged at the beginning, a double dash separator, and then the message. This format corresponds to the format argument we used in basicConfig.

# Later, we'll be parsing these each message and use the time as a piece of data to explore.

# Continuous message writing
# Now on to continuously checking for new messages in a loop.

# When we're connected to the socket, Twitch (and other IRC) will periodically send a keyword — "PING" — to check if you're still using the connection. We want to check for this keyword, and send an appropriate response — "PONG".

# One other thing we'll do is parse emojis so they can be written to a file. To do this, we'll use the emoji library that will provide a mapping from emojis to their meaning in words. For example, if a 👍 shows up in a message it'll be converted to :thumbs_up:.

# The following is a while loop that will continuously check for new messages from the socket, send a PONG if necessary, and log messages with parsed emojis:

from emoji import demojize

counter=1

while True:
    resp = sock.recv(2048).decode('utf-8')
    print(resp)
    
    if resp.startswith('PING'):
        sock.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        logging.info(demojize(resp))
    
    print(counter)
    counter+=1
        
# This will keep running until you stop it. To see the messages in real-time open a new terminal, navigate to the log's location, and run tail -f chat.log.

:techmaster11!techmaster11@techmaster11.tmi.twitch.tv PRIVMSG #disguisedtoast :EZ Clap
:iamnatedunn!iamnatedunn@iamnatedunn.tmi.twitch.tv PRIVMSG #disguisedtoast :toxicToast
:arcanesandman!arcanesandman@arcanesandman.tmi.twitch.tv PRIVMSG #disguisedtoast :Conceded Pog
:plusplushs!plusplushs@plusplushs.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL
:master_dodge!master_dodge@master_dodge.tmi.twitch.tv PRIVMSG #disguisedtoast :hahaahhahahahahah
:jomilk!jomilk@jomilk.tmi.twitch.tv PRIVMSG #disguisedtoast :EZ Clap
:alphagriffin!alphagriffin@alphagriffin.tmi.twitch.tv PRIVMSG #disguisedtoast :WINNING Pog
:theropetosser!theropetosser@theropetosser.tmi.twitch.tv PRIVMSG #disguisedtoast :EZ Clap
:pokedesu!pokedesu@pokedesu.tmi.twitch.tv PRIVMSG #disguisedtoast :EZ Clap
:fusrodaha!fusrodaha@fusrodaha.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL
:00hadock!00hadock@00hadock.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL
:swampmaster12345!swampmaster12345@swampmaster12345.tmi.twitch.tv PRIVMSG #disguisedtoa

:2020v1s1on!2020v1s1on@2020v1s1on.tmi.twitch.tv PRIVMSG #disguisedtoast :ITS GODZILLA

64
:connorkillz!connorkillz@connorkillz.tmi.twitch.tv PRIVMSG #disguisedtoast :Just like all of twitch

65
:kingofneverland123!kingofneverland123@kingofneverland123.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL

66
:doory!doory@doory.tmi.twitch.tv PRIVMSG #disguisedtoast :the kids screams will haunt me

67
:hs_sirsalty!hs_sirsalty@hs_sirsalty.tmi.twitch.tv PRIVMSG #disguisedtoast :Louder pepeD

68
:theropetosser!theropetosser@theropetosser.tmi.twitch.tv PRIVMSG #disguisedtoast :REEEEEEEEE

69
:ghostbulla!ghostbulla@ghostbulla.tmi.twitch.tv PRIVMSG #disguisedtoast :whos that?

70
:plusplushs!plusplushs@plusplushs.tmi.twitch.tv PRIVMSG #disguisedtoast :LOL

71
:theropetosser!theropetosser@theropetosser.tmi.twitch.tv PRIVMSG #disguisedtoast :toastyREE1 toastyREE2 toastyREE3

72
:newfom!newfom@newfom.tmi.twitch.tv PRIVMSG #disguisedtoast :LULW

73
:jomilk!jomilk@jomilk.tmi.twitch.tv PRIVMSG #disguisedtoast 

:silverskullshd!silverskullshd@silverskullshd.tmi.twitch.tv PRIVMSG #disguisedtoast :have you heard the thanos remix of old town road LUL

152
:running_pixels!running_pixels@running_pixels.tmi.twitch.tv PRIVMSG #disguisedtoast :im confused do i TriKool or do i KKool

153
:krocdp!krocdp@krocdp.tmi.twitch.tv PRIVMSG #disguisedtoast :!rank

154
:nightbot!nightbot@nightbot.tmi.twitch.tv PRIVMSG #disguisedtoast :DisguisedToast69's current rank: Silver II https://na.op.gg/summoner/userName=DisguisedToast69

155
:dakillawon13!dakillawon13@dakillawon13.tmi.twitch.tv PRIVMSG #disguisedtoast :pepeJam

156
:cyan4010!cyan4010@cyan4010.tmi.twitch.tv PRIVMSG #disguisedtoast :for a 4500 viewer, your chat is so calm

157
:childish_flaming0!childish_flaming0@childish_flaming0.tmi.twitch.tv PRIVMSG #disguisedtoast :Mods can’t tell me nothing monkaS

158
:bluempbau!bluempbau@bluempbau.tmi.twitch.tv PRIVMSG #disguisedtoast :Red dead redemption

159
:sonnyd50!sonnyd50@sonnyd50.tmi.twitch.tv PRIVMSG #disgui

:rafael0515!rafael0515@rafael0515.tmi.twitch.tv PRIVMSG #disguisedtoast :AYAYA

229
:rebelmytzu!rebelmytzu@rebelmytzu.tmi.twitch.tv PRIVMSG #disguisedtoast :AYAYA

230
:rawkclayton!rawkclayton@rawkclayton.tmi.twitch.tv PRIVMSG #disguisedtoast :AYAYA

231
:kyusho!kyusho@kyusho.tmi.twitch.tv PRIVMSG #disguisedtoast :D:

232
:professor_kkona!professor_kkona@professor_kkona.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

233
:apolo013!apolo013@apolo013.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL

234
:demonw0lf!demonw0lf@demonw0lf.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

235
:ededu27!ededu27@ededu27.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

236
:enrifh!enrifh@enrifh.tmi.twitch.tv PRIVMSG #disguisedtoast :pogg

237
:yummychicken!yummychicken@yummychicken.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

238
:el_hawkeye!el_hawkeye@el_hawkeye.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

239
:hs_sirsalty!hs_sirsalty@hs_sirsalty.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

240
:legitmonkeyy!legitmonkey

:tyragi!tyragi@tyragi.tmi.twitch.tv PRIVMSG #disguisedtoast :I somehow prefer this

319
:tungdil007!tungdil007@tungdil007.tmi.twitch.tv PRIVMSG #disguisedtoast :@DisguisedToast we cant see what your doing! please open your lol window

320
:n3vr!n3vr@n3vr.tmi.twitch.tv PRIVMSG #disguisedtoast :who is this guy Pog

321
:axelflo55!axelflo55@axelflo55.tmi.twitch.tv PRIVMSG #disguisedtoast :it cause it full screen application and not windowed friend

322
:untraceablesauce!untraceablesauce@untraceablesauce.tmi.twitch.tv PRIVMSG #disguisedtoast :monkaS

323
:guanfari!guanfari@guanfari.tmi.twitch.tv PRIVMSG #disguisedtoast :penta kill pog

324
:kalonaa45!kalonaa45@kalonaa45.tmi.twitch.tv PRIVMSG #disguisedtoast :gachiBASS

325
:murdurburd!murdurburd@murdurburd.tmi.twitch.tv PRIVMSG #disguisedtoast :BLIND FRIENDLY STREAM Pog

326
:bewedee!bewedee@bewedee.tmi.twitch.tv PRIVMSG #disguisedtoast :This is the quality gameplay i came for

327
:axelflo55!axelflo55@axelflo55.tmi.twitch.tv PRIVMSG #disg

:swampmaster12345!swampmaster12345@swampmaster12345.tmi.twitch.tv PRIVMSG #disguisedtoast :toast has outjungled a challenger b4 so...

396
PING :tmi.twitch.tv

397
:pc__adorable!pc__adorable@pc__adorable.tmi.twitch.tv PRIVMSG #disguisedtoast :is this ranked?

398
:bluelizard75!bluelizard75@bluelizard75.tmi.twitch.tv PRIVMSG #disguisedtoast :Tf has the control lytMOTIVATED

399
:embronoel1!embronoel1@embronoel1.tmi.twitch.tv PRIVMSG #disguisedtoast :if he playing solo ?¡

400
:sushismystic!sushismystic@sushismystic.tmi.twitch.tv PRIVMSG #disguisedtoast :wats ur opinion on crazy morde rework @toast?

401
:uzula_!uzula_@uzula_.tmi.twitch.tv PRIVMSG #disguisedtoast :since when did he change his mic

402
:plusplushs!plusplushs@plusplushs.tmi.twitch.tv PRIVMSG #disguisedtoast :not even close baby

403
:trivex_!trivex_@trivex_.tmi.twitch.tv PRIVMSG #disguisedtoast :gachiBASS

404
:chunkymonkey42!chunkymonkey42@chunkymonkey42.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL

405
:lovelypleb!lovelypl

:faultywirestv!faultywirestv@faultywirestv.tmi.twitch.tv PRIVMSG #disguisedtoast :!Rank

482
:junqlejim!junqlejim@junqlejim.tmi.twitch.tv PRIVMSG #disguisedtoast :disguisedtoast IS THAT GROSS GORE??? Kappa

483
:nightbot!nightbot@nightbot.tmi.twitch.tv PRIVMSG #disguisedtoast :DisguisedToast69's current rank: Silver II https://na.op.gg/summoner/userName=DisguisedToast69

484
:cakehp!cakehp@cakehp.tmi.twitch.tv PRIVMSG #disguisedtoast :@DisguisedToast if you had to eat one type of food for the rest of your life, what would it be?

485
:zorrostylex3!zorrostylex3@zorrostylex3.tmi.twitch.tv PRIVMSG #disguisedtoast :Hose song pls BibleThump Hose song pls BibleThump Hose song pls BibleThump

486
:carlosedu7781!carlosedu7781@carlosedu7781.tmi.twitch.tv PRIVMSG #disguisedtoast :thx

487
:bruticyp!bruticyp@bruticyp.tmi.twitch.tv PRIVMSG #disguisedtoast :@JuicyJovan gachiBASS

488
:plusplushs!plusplushs@plusplushs.tmi.twitch.tv PRIVMSG #disguisedtoast :WutFace

489
:briankhoi!briankhoi@briankhoi

:selvin369!selvin369@selvin369.tmi.twitch.tv PRIVMSG #disguisedtoast :monkaW

569
:bruticyp!bruticyp@bruticyp.tmi.twitch.tv PRIVMSG #disguisedtoast :EZ ULT LUL

570
:el_hawkeye!el_hawkeye@el_hawkeye.tmi.twitch.tv PRIVMSG #disguisedtoast :HOLY

571
:dr_device!dr_device@dr_device.tmi.twitch.tv PRIVMSG #disguisedtoast :monkaS

572
:chewypi!chewypi@chewypi.tmi.twitch.tv PRIVMSG #disguisedtoast :monkaW

573
:templarre!templarre@templarre.tmi.twitch.tv PRIVMSG #disguisedtoast :monkaW

574
:th1122!th1122@th1122.tmi.twitch.tv PRIVMSG #disguisedtoast :monkaW

575
:plusplushs!plusplushs@plusplushs.tmi.twitch.tv PRIVMSG #disguisedtoast :monka

576
:lovelypleb!lovelypleb@lovelypleb.tmi.twitch.tv PRIVMSG #disguisedtoast :he could got you too

577
:chunkymonkey42!chunkymonkey42@chunkymonkey42.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL

578
:antosha_steeelseries!antosha_steeelseries@antosha_steeelseries.tmi.twitch.tv PRIVMSG #disguisedtoast :LUL

579
:selvin369!selvin369@selvin369.tmi.twitch.tv PRIVM

:junqlejim!junqlejim@junqlejim.tmi.twitch.tv PRIVMSG #disguisedtoast :Ez clap

640
:lovelypleb!lovelypleb@lovelypleb.tmi.twitch.tv PRIVMSG #disguisedtoast :@bruticyp yes

641
:bigbohja!bigbohja@bigbohja.tmi.twitch.tv PRIVMSG #disguisedtoast :Do viewer games pog

642
:plusplushs!plusplushs@plusplushs.tmi.twitch.tv PRIVMSG #disguisedtoast :trundle ult on nasus monkaS

643
:bruticyp!bruticyp@bruticyp.tmi.twitch.tv PRIVMSG #disguisedtoast :@lovelypleb Is that like a new meta? or a new silver meta?

644
:deathblok!deathblok@deathblok.tmi.twitch.tv PRIVMSG #disguisedtoast :Pog

645
:wyvernite!wyvernite@wyvernite.tmi.twitch.tv PRIVMSG #disguisedtoast :Toast actually 1v5 carrying his boosted team

646
:templarre!templarre@templarre.tmi.twitch.tv PRIVMSG #disguisedtoast :chocoNT

647
:purelybaka!purelybaka@purelybaka.tmi.twitch.tv PRIVMSG #disguisedtoast :POG

648
:zegnergaming!zegnergaming@zegnergaming.tmi.twitch.tv PRIVMSG #disguisedtoast :yikes

649
:norickh!norickh@norickh.tmi.twitch.tv PRI

In [11]:
#close socket IF DONE

sock.close()